In [41]:
import torch
from dlc_practical_prologue import generate_pair_sets
import torch.nn as nn

In [42]:
train_input_0, train_target_0, train_classes_0, test_input_0, test_target_0, test_classes_0 = generate_pair_sets(1000)

In [43]:
def normalize_data(tensor):
    mu, std = tensor.mean(), tensor.std()
    tmp = tensor.sub(mu).div(std)
    return tmp

In [44]:
def convert_to_one_hot_labels(target):
    tmp = target.new_zeros(target.size(0), target.max() + 1)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

In [45]:
train_input = normalize_data(train_input_0)
# train_target = convert_to_one_hot_labels(train_target_0)
train_target = train_target_0

test_input = normalize_data(test_input_0)
# test_target = convert_to_one_hot_labels(test_target_0)
test_target = test_target_0

In [46]:
print(train_input.mean())
print(train_input.std())

tensor(-3.7104e-07)
tensor(1.0000)


In [47]:
print(test_input.mean())
print(test_input.std())

tensor(1.6006e-07)
tensor(1.0000)


In [136]:
import models
import torch.nn.functional as F

In [144]:
class BaselineNetwork2(nn.Module):
    def __init__(self):
        super(BaselineNetwork2, self).__init__()
        self.fc1 = nn.Linear(2 * 14 * 14, 2)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

In [223]:
basic_model = models.BaselineNetwork()

In [229]:
def train_model(model, train_input, train_target, train_classes):
    lr, nb_epochs, batch_size = 1e-1, 20, 100 
    
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    for e in range(nb_epochs):
        for inputs, targets in zip(train_input.split(batch_size),
                                  train_target.split(batch_size)):
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [230]:
train_model(basic_model, train_input, train_target, train_classes_0)

In [231]:
BATCH_SIZE = 10
LOG_INTERVAL = 20

In [232]:
def test(test_input, test_target, test_classes, model, criterion):
    with torch.no_grad():
        nb_data_errors = 0
        loss_sum = 0
        for b in range(0, test_input.size(0), BATCH_SIZE):
            output = model(test_input.narrow(0, b, BATCH_SIZE))
            loss = criterion(output, test_target.narrow(0, b, BATCH_SIZE))
            loss_sum += loss
            _, predicted_classes = torch.max(output, 1)

            for k in range(BATCH_SIZE):
                #print(repr(test_target[b + k]) + " - " + repr(predicted_classes[k]))
                if test_target[b + k] != predicted_classes[k]:
                    nb_data_errors = nb_data_errors + 1

            if b % LOG_INTERVAL == 0:
                pass
                #print("Accuracy: " + repr())
                #print_test(batch_idx, len(val_loader), batch_time, losses, top1, persistent=False, color=color, title=title)
    
        print("Nb errors: " + repr(nb_data_errors))
        accuracy = 1 - (nb_data_errors / test_input.size(0))
        print("Accuracy: " + repr(accuracy * 100) + "%" + " - Loss: " + repr(loss_sum.item()))

In [233]:
test(test_input, test_target, test_classes_0, basic_model, nn.CrossEntropyLoss())

Nb errors: 192
Accuracy: 80.80000000000001% - Loss: 127.9828109741211
